# Fuzzy Based Shunt Power Active Filter

## **Plan Of Attack**

### **Understanding Role of Fuzzy in SPAF Setup**

- _Used as DC-Link Capacitor Voltage Algorithm_
- _Capacitor Voltage, `Vc` is taken as `14v`, ( [refer ijsert page6](http://www.ijesrt.com/issues%20pdf%20file/Archive-2018/June-2018/64.pdf) )_
- _A voltage is measured after the shunt and before Fuzzy, `Vm`_
- _Resistor of `0.1 Ohm` is taken due to impedence_


### **Crafting Fuzzy Sets**

**_Input Set_**

There are two sets: _Set1, Set2_

*Set1* : Set of errors (`E`) are collected between `Vc` and `Vm` at a specific point of time `t`, given by:

```bash
                                   E = Vc - Vm
```

*Set2* : Set of errors described by (`dE`):

```bash
                                dE = E(t) - E(t-1)
                                
                                        or
                                        
                                dE = E - d(E)/dt

```

**_Output Set_**

The output set is a crisp value (percentage) of the current produced with effective magnitude: `Ic`. The range of `Ic` can be easily calculated from `Vc` assuming the shunt has a resistance of `0.1 Ohm` (caused by impedance).

Since the `E` range is from 7 to 14, `Vc` is 14V, `Ic` can be calculated by:

```bash
                                Ic = x ∈ {(Vc-Vm) / R}
```

**Fuzzy Set Ranges**

_E Range_:

The experimentally permitted range is : `E = {7 to 14}`

_dE Range_:

The expereimentally feasible range is : `dE = {-0.4 to 0.5}`

_Ic Range_:

The range acquired by calculation from above : `Ic = {0 to 70}`

> All ranges are being refered from [ijsert-page3](http://www.ijesrt.com/issues%20pdf%20file/Archive-2018/June-2018/64.pdf)

### **Chosing Fuzzy Inference System**

Using _SKFuzzy `control.ControlSystem`_ which defaults to _Mamdani Inference System_ with defuzzification using _centroid_

### **Coding**

- _Defining universe_
- _Initializing Antecedents, Consequents fuzzy variables_
- _Populate the universe with membership functions_
- _Rule Writing_
- _Control System + Simulation_

#### main

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:

#E = np.linspace(7,14)
#dE = np.linspace(-0.4,0.4)

Vc = 14
R = 0.1

# inputs
E = ctrl.Antecedent(np.arange(7,14), 'Error')
dE = ctrl.Antecedent(np.linspace(-0.4, 0.5), 'Rate of Error')
print(dE)

# output

Ic_minimum = (Vc - 14) / 0.1      # when capacitor is not charged initially
Ic_maximum = (Vc - 7) / 0.1       # when capacitor is full charged
Ic = ctrl.Consequent(np.arange(Ic_minimum, Ic_maximum), 'Current Output', defuzzify_method='centroid')

# membership functions
memberships = ['NL','NM','NS','EZ','PS','PM','PL']

# for E
E.automf(names=memberships)

# for dE
dE.automf(names = memberships)

# for Ic
Ic.automf(names=memberships)
Ic['NL'] = fuzz.trapmf(Ic.universe, [0,0,5,10])
Ic['PL'] = fuzz.trapmf(Ic.universe, [60,65,70,70]) # refer ijsert

# view membership graphs
E.view()
dE.view()
Ic.view()


#### rule writing

Since we have a pair of `Antecedents` and one `Consequent` of 7 memberships each, We will have to write a proper 7 rules connecting `Antecedent` to `Consequent` properly.

Rules have been provided in a tabular form in the [scholarly article](https://moscow.ggws.net/2159/5f5f49fbfb670fca04480df52ac3ce71/mekri2006.pdf#navpanes=0&view=FitH)

In [ ]:
rule0 = ctrl.Rule(antecedent=( (dE['NL'] & E['NL']) |
                               (dE['NL'] & E['NM']) |
                               (dE['NL'] & E['NS']) |
                               (dE['NL'] & E['EZ']) |
                               (dE['NM'] & E['NL']) |
                               (dE['NM'] & E['NM']) |
                               (dE['NM'] & E['NS']) |
                               (dE['NS'] & E['NL']) |
                               (dE['NS'] & E['NM']) |
                               (dE['EZ'] & E['NL']) 
                             ), consequent=Ic['NL'])

rule1 = ctrl.Rule(antecedent=( (dE['NL'] & E['PS']) |
                               (dE['NM'] & E['EZ']) |
                               (dE['NS'] & E['NS']) |
                               (dE['EZ'] & E['NM']) |
                               (dE['PS'] & E['NL']) 
                             ), consequent=Ic['NM'])

rule2 = ctrl.Rule(antecedent=( (dE['NL'] & E['PM']) |
                               (dE['NM'] & E['PS']) |
                               (dE['NS'] & E['EZ']) |
                               (dE['EZ'] & E['NS']) |
                               (dE['PS'] & E['NM']) |
                               (dE['PM'] & E['NL']) 
                             ), consequent=Ic['NS'])

rule3 = ctrl.Rule(antecedent=( (dE['NL'] & E['PL']) |
                               (dE['NM'] & E['PM']) |
                               (dE['NS'] & E['PS']) |
                               (dE['EZ'] & E['EZ']) |
                               (dE['PS'] & E['NS']) |
                               (dE['PM'] & E['NM']) |
                               (dE['PL'] & E['NL']) 
                             ), consequent=Ic['EZ'])

rule4 = ctrl.Rule(antecedent=( (dE['PL'] & E['NM']) |
                               (dE['PM'] & E['NS']) |
                               (dE['PS'] & E['EZ']) |
                               (dE['EZ'] & E['PS']) |
                               (dE['NS'] & E['PM']) |
                               (dE['NM'] & E['PL']) 
                             ), consequent=Ic['PS'])                                                          

rule5 = ctrl.Rule(antecedent=( (dE['PL'] & E['NS']) |
                               (dE['PM'] & E['EZ']) |
                               (dE['PS'] & E['PS']) |
                               (dE['EZ'] & E['PM']) |
                               (dE['NS'] & E['PL']) 
                             ), consequent=Ic['PM'])

rule6 = ctrl.Rule(antecedent=( (dE['PL'] & E['PL']) |
                               (dE['PL'] & E['PM']) |
                               (dE['PL'] & E['PS']) |
                               (dE['PL'] & E['EZ']) |
                               (dE['PM'] & E['PL']) |
                               (dE['PM'] & E['PM']) |
                               (dE['PM'] & E['PS']) |
                               (dE['PS'] & E['PL']) |
                               (dE['PS'] & E['PM']) |
                               (dE['EZ'] & E['PL']) 
                             ), consequent=Ic['NL'])

#### view rules

In [ ]:
views = [rule0,rule1,rule2,rule3,rule4,rule5,rule6]

for i in views:
    i.view()

#### control system + simulation

In [ ]:
Ic_control_system = ctrl.ControlSystem([rule0,rule1,rule2,rule3,rule4,rule5,rule6])

Ic_output_simulation = ctrl.ControlSystemSimulation(Ic_control_system)

Ic_output_simulation.inputs({'Error': 12, 'Rate of Error': -0.1}) #11.4, -0.1
Ic_output_simulation.compute()

print(Ic_output_simulation.output['Current Output'])
Ic.view(sim=Ic_output_simulation)

## **References**

- mdpi-res.com/electronics
- ijesrt scholarly article
- moscow.ggws.net